In [1]:
import pandas as pd
from scipy.stats import norm
import statistics
import math
import numpy as np
from scipy.stats import t

In [2]:
dados = pd.read_csv("BD01.csv")
display(dados)

,Fornecedor,Degradacao,TempoFalha,nReincidenciaFalhas
0,B,Alto,"1,957,317,971",2
1,B,Alto,0.364676956,13
2,B,Medio,"2,640,281,946",1
3,B,Alto,"1,187,133,811",0
4,A,Baixo,"1,833,797,289",4
...,...,...,...,...
574,B,Medio,"3,317,492,501",12
575,A,Baixo,"1,520,280,108",3
576,A,Baixo,"3,408,626,944",22
577,A,Baixo,"2,016,019,969",5


## 0. Funções

In [3]:
def AmostraAleatoriaSimples(dataframe, n):
    amostra = dataframe.sample(n, replace = False)
    return amostra

In [4]:
def AmostraAleatoriaSistematica(dataframe,n):
    N = len(dataframe)
    ti = 0
    tf = (N-1)
    d = round((tf - ti + 1)/n)
    i0 = np.random.default_rng().integers(low=ti, high=ti+d)
    sampleIndexes = i0 + list(range(0, N, d))
    sampleData = dataframe.loc[sampleIndexes, ]
    print("*********** sampleData ***********")
    return (sampleData)

In [5]:
def getProportionalStratifiedRandomSample(dataframe,n, varName,subSampleFunction =AmostraAleatoriaSimples):
    # distribuição da variável de estrato na população
    probDist = dataframe[varName].value_counts(normalize=True) # View(probDist)
    print('*************** probDist ***************')
    display(probDist)
    # ns = round(n*probDist.values);  # View(ns)
    stratums = probDist.index.to_list()
    nStratums = len(stratums)
    sampleData = pd.DataFrame()
    for i in range(nStratums):
        stratum_i = stratums[i]
        # View(dataFrame_i, title = stratum_i)
        dataFrame_i = dataframe[dataframe[varName] == stratum_i]
        N_i = len(dataFrame_i)  # tamanho da iª população
        n_i = round(n*probDist.values[i])  # tamanho da iª amostra
        # View(sampleData_i, title = paste("amostra_", stratum_i) )
        sampleData_i = subSampleFunction(dataframe= dataFrame_i, n= n_i)
        #sampleData_i = getSystematicRandomSample(dataFrame= dataFrame_i, sampleSize= n_i)
        sampleData = pd.concat([sampleData, sampleData_i], axis = 0)
    return (sampleData)

In [6]:
def DescribeAmostra(dataframe,amostra):
    n = len(amostra)
    display(amostra)
    print("--------------- Descrição dos dados ---------------")
    display(dataframe.describe().transpose())
    print("--------------- Descrição da amostra ---------------")
    display(amostra.describe().transpose())

In [7]:
def EstimativaPont(dataframe,amostra,n_vezes_amo,n_vezes_dado):
    tam_Amos = len(amostra)
    tam_dados = len(dataframe)
    probabilidade_amostra = n_vezes_amo/tam_Amos
    probabilidade_dados =  n_vezes_dado/tam_dados
    print("------------Probabilidade na Amostra ------------") 
    display(probabilidade_amostra)
    print("------------Probabilidade na População Total ------------")  
    display(probabilidade_dados)  

In [8]:
def Intervalo_Confianca(dados,amostra,intervalo_de_confiança, n_vezes_amo):
    N = len(dados)
    n = len(amostra)
    prob_Amos = n_vezes_amo/n
    #olhar conservador
    sd = np.sqrt(prob_Amos*(1-prob_Amos))
    fator_de_cor = np.sqrt((N-n)/(N-1))
    sd_prob_Amos  = (sd/np.sqrt(n))*fator_de_cor
    z = norm.ppf(1-(1- intervalo_de_confiança)/2) 
    lower = prob_Amos - z*sd_prob_Amos 
    upper = prob_Amos + z*sd_prob_Amos
    lower = round(lower, 5) if lower >= 0 else 0
    upper = round(upper, 5) if upper <=1 else 1
    print(f"[  {lower}  ,  {upper}  ]")

In [9]:
def Intervalo_Confianca_T(dados,amostra,intervalo_de_confiança,n_vezes_amo):
    N = len(dados)
    n = len(amostra)
    prob_Amos = n_vezes_amo/n
    sd = np.sqrt(prob_Amos*(1-prob_Amos))
    sd_prob_Amos  = (sd/np.sqrt(N))
    v_1_alpha_2 = 1-(1-intervalo_de_confiança)/2; 
    tk = round(t.ppf(v_1_alpha_2, df=(n-1)), 3)
    correctionFactor = ((N-n)/(N-1))
    sd_prob_Amos = np.sqrt(((sd*sd)/(n))*correctionFactor)
    lower = prob_Amos - tk*sd_prob_Amos 
    upper = prob_Amos + tk*sd_prob_Amos
    lower = round(lower, 5) if lower >= 0 else 0
    upper = round(upper, 5) if upper <=1 else 1
    print(f"[  {lower}  ,  {upper}  ]")

## FUNCOES MED

In [10]:
def Intervalo_Confianca_Media(dados,amostra,varName, intervalo_de_confiança):
    N = len(dados)
    n = len(amostra)
    med_amostra = np.mean(amostra[varName])
    sd = np.std(dados[varName])
    fator_de_cor = np.sqrt((N-n)/(N-1)) if N is not None else 1
    sd_prob_Amos  = (sd/np.sqrt(n))*fator_de_cor
    # t estudant se desvio padrao da amostra
    z = norm.ppf(1-(1- intervalo_de_confiança)/2) 
    lower = med_amostra - z*sd_prob_Amos 
    upper = med_amostra + z*sd_prob_Amos 
    lower = round(lower, 5)
    upper = round(upper, 5)
    print(f"[  {lower}  ,  {upper}  ]")
    
#CHECADO

In [11]:
def Intervalo_Confianca_Media_T (dados,amostra,intervalo_de_confianca):
    N = len(dados)
    n = len(amostra)
    med_amostra = np.mean(amostra['nReincidenciaFalhas'])
    sd = np.std(amostra['nReincidenciaFalhas'])
    v_1_alpha_2 = 1-(1-intervalo_de_confianca)/2; 
    tk = round(t.ppf(v_1_alpha_2, df=(n-1)), 3)
    correctionFactor = ((N-n)/(N-1))
    sd_prob_Amos = np.sqrt(((sd*sd)/(n))*correctionFactor)
    lower = med_amostra- tk*sd_prob_Amos 
    upper = med_amostra + tk*sd_prob_Amos 
    lower = round(lower, 5) 
    upper = round(upper, 5)
    print(f"[  {lower}  ,  {upper}  ]")
#CHECADO    

In [12]:
def EstimativaPontMed(dataframe, amostra, varName):
    med_Amos = np.mean(amostra[varName])
    med_Dados = np.mean(dataframe[varName])
    print("------------Média na Amostra ------------") 
    display(med_Amos)
    print("-----------Média na População Total ------------")  
    display(med_Dados)  
    
#CHECADO    

## 1. Amostra aleatória simples

* Analisando a nossa  amostra aleatoóia simples vemos que ela não representa tão bem nossa população inteira.  
Na amostra temos valores de média em 26.1 enquanto na populção temos o valor de média de 18.79 uma diferença de mais de 7 pontos o que representa uma diferença alta.  
Outra grande diferença observada é no desvio padrão, na amostra o desvio padrão foi menor do que na população total.

In [13]:
amostra = AmostraAleatoriaSimples(dados, 10)
DescribeAmostra(dados,amostra)

,Fornecedor,Degradacao,TempoFalha,nReincidenciaFalhas
461,A,Baixo,"145,020,921",37
361,B,Medio,"3,978,507,524",39
406,A,Baixo,"7,801,743,213",3
429,A,Baixo,"2,440,084,055",30
134,B,Medio,"4,757,678,813",47
575,A,Baixo,"1,520,280,108",3
390,B,Alto,0.123488333,21
425,A,Baixo,"2,015,703,222",0
239,A,Baixo,"418,140,378",53
431,A,Baixo,"1,412,206,249",28


--------------- Descrição dos dados ---------------


,count,mean,std,min,25%,50%,75%,max
nReincidenciaFalhas,579.0,18.792746,19.226708,0.0,3.0,14.0,28.0,125.0


--------------- Descrição da amostra ---------------


,count,mean,std,min,25%,50%,75%,max
nReincidenciaFalhas,10.0,26.1,18.970445,0.0,7.5,29.0,38.5,53.0


* Ao calcularmos a probabilidade do fornecedor ser do tipo A tanto na amostra quanto na população geral também observamos diferenças.  
A probabilidade na amostra é de 70%, uma probabilidade alta de ocorrer enquanto na população total é de 60.97% uma probabilidade menor que na amostra, quase 10% de diferença.

In [14]:
n_vezes_amo_A,n_vezes_amo_B = amostra['Fornecedor'].value_counts()
n_vezes_dados_A,n_vezes_dados_B = dados['Fornecedor'].value_counts()
EstimativaPont(dados,amostra,n_vezes_amo_A,n_vezes_dados_A)

------------Probabilidade na Amostra ------------


0.7

------------Probabilidade na População Total ------------


0.6096718480138169

* Ao calcularmos o intervalo de confiança vemos intervalos altamente espaçados, o intervalo na amostra é menor e nossa probabilidade se encaixa nesse intervalo, porém no intervalo da população vemos um intervalo muito grande que também contempla nossa probabilidade mas com um espaçamento muito grande.

In [15]:
print("\n")
print("------------------ Intervalo na Amostra-----------------")
Intervalo_Confianca(dados,amostra,0.95,n_vezes_amo_A)
print("\n")
print("------------------ Intervalo na População Total-----------------")
Intervalo_Confianca_T(dados,amostra,0.95,n_vezes_amo_A)




------------------ Intervalo na Amostra-----------------
[  0.41819  ,  0.98181  ]


------------------ Intervalo na População Total-----------------
[  0.37477  ,  1  ]


## 2. Amostra aleatória Sistematica

* Já na amostra aleatória Sistematica vemos uma diferença menor na descrição da amostra em relação aos dados populacionais, a média na amostra é de 16 e o desvio padrão é de 23.37, já na população total obsefvamos uma média maior em torno de 18.79 e um desvio padrão menor em torno de 19.23.

In [16]:
amostra2 = AmostraAleatoriaSistematica(dados,10)
DescribeAmostra(dados,amostra2)


*********** sampleData ***********


,Fornecedor,Degradacao,TempoFalha,nReincidenciaFalhas
2,B,Medio,"2,640,281,946",1
60,A,Baixo,"6,443,583,406",1
118,B,Medio,"3,918,342,738",4
176,A,Baixo,"5,391,756,364",47
234,A,Baixo,"7,560,281,259",66
292,A,Baixo,"8,061,834,509",5
350,A,Baixo,"3,206,266,297",29
408,B,Medio,"4,706,314,575",0
466,A,Baixo,"2,122,048,953",4
524,B,Alto,0.408561652,3


--------------- Descrição dos dados ---------------


,count,mean,std,min,25%,50%,75%,max
nReincidenciaFalhas,579.0,18.792746,19.226708,0.0,3.0,14.0,28.0,125.0


--------------- Descrição da amostra ---------------


,count,mean,std,min,25%,50%,75%,max
nReincidenciaFalhas,10.0,16.0,23.366643,0.0,1.5,4.0,23.0,66.0


* Ao verificarmos a probabilidade de o nivel de degradação ser alto vemos que elas não estão extremamente distantes, a probabilidade na amostra é de 10% enquanto na população total é de 19.52% aproximadamente 10% maior que na amostra.  


In [17]:

n_amostra_baixo,n_amostra_medio,n_amostra_alto = amostra2['Degradacao'].value_counts()
n_dados_baixo,n_dados_medio,n_dados_alto = dados['Degradacao'].value_counts()
EstimativaPont(dados,amostra2,n_amostra_alto,n_dados_alto)



------------Probabilidade na Amostra ------------


0.1

------------Probabilidade na População Total ------------


0.19516407599309155

* Ao verificarmos o intervalo de confiança vemos que eles estão bem parecidos.  
Os intervalos tem somente 0,03 de difernça na parte superior e ambos contemplam suas probabilidades.

In [18]:
print("\n")
print("------------------ Intervalo na Amostra-----------------")
Intervalo_Confianca(dados,amostra2,0.95, n_amostra_alto)
print("\n")
print("------------------ Intervalo na População Total-----------------")
Intervalo_Confianca_T(dados,amostra2,0.95,n_amostra_alto)



------------------ Intervalo na Amostra-----------------
[  0  ,  0.28449  ]


------------------ Intervalo na População Total-----------------
[  0  ,  0.31291  ]


## 3. Amostra Simples Estrtificada

* Na amostra Simples Estrtificada observamos valores bem diferentes e distantes da população geral.
Os valores de média e desvio padrão na amostra são bem menores.  
Na amostra temos um valor de 14.5 de média enquanto na população temos 18.79 de média uma diferença de mais de 4 pontos.
Tambem temos na amostra temos um valor de 12.53 de desvio padrão enquanto na população temos 19.23 de desvio padrão uma diferença de aproximadamente 7 pontos.

In [19]:
amostra3 =getProportionalStratifiedRandomSample(dataframe=dados, n=10, varName='Fornecedor',subSampleFunction = AmostraAleatoriaSimples)
DescribeAmostra(dados,amostra3)

*************** probDist ***************


Fornecedor
A    0.609672
B    0.390328
Name: proportion, dtype: float64

,Fornecedor,Degradacao,TempoFalha,nReincidenciaFalhas
507,A,Baixo,"118,429,296",1
265,A,Baixo,"1,399,006,623",3
245,A,Baixo,"314,363,297",22
426,A,Baixo,"561,931,698",0
401,A,Baixo,"1,065,649,149",39
133,A,Baixo,"1,022,324,181",17
306,B,Alto,0.522407689,13
257,B,Alto,0.692945334,5
30,B,Alto,0.548219478,21
381,B,Medio,"2,457,423,411",24


--------------- Descrição dos dados ---------------


,count,mean,std,min,25%,50%,75%,max
nReincidenciaFalhas,579.0,18.792746,19.226708,0.0,3.0,14.0,28.0,125.0


--------------- Descrição da amostra ---------------


,count,mean,std,min,25%,50%,75%,max
nReincidenciaFalhas,10.0,14.5,12.527747,0.0,3.5,15.0,21.75,39.0


* Ao anarlisarmos o número médio de reincidência de falhas vemos uma grande diferença nesses valores.  
a média na Amostra é de 14.5 enqunto a média na População Total é de 18.79 o qu representa uma diferença de aproximadamente 4 pontos.

In [20]:
EstimativaPontMed(dados, amostra3,"nReincidenciaFalhas")

------------Média na Amostra ------------


14.5

-----------Média na População Total ------------


18.792746113989637

* Já ao verificarmos o intervalo de confiança vemos que eles estão extremamente espaçados, porém que contemplam as medias na amostra e na populção total.  
O intervalo de confiança da população total apresenta-se em um melhor estado por ser de tamanho menor.

In [21]:

#checado
print("\n")
print("------------------ Intervalo na Amostra-----------------")
Intervalo_Confianca_Media(dados,amostra3,"nReincidenciaFalhas", 0.95)
print("\n")
print("------------------ Intervalo na População Total-----------------")
Intervalo_Confianca_Media_T (dados,amostra3,0.95)




------------------ Intervalo na Amostra-----------------
[  2.68674  ,  26.31326  ]


------------------ Intervalo na População Total-----------------
[  6.06512  ,  22.93488  ]
